### Batch mySQL to Spark (1)

In [9]:
import pandas as pd
from pyspark.sql import SparkSession

## Connect to MySQL (Airport)
dataframe_mysql_airport = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://127.0.0.1 /skyscanner_batch")\
.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "airports").option("user", "root")\
.option("password", "").load()

## Connect to MySQL (Cities)
dataframe_mysql_cities = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://127.0.0.1 /skyscanner_batch")\
.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "cities").option("user", "root")\
.option("password", "").load()


In [10]:
#Airport Table
AirportDF =  dataframe_mysql_airport.toPandas()
AirportJSON = AirportDF.to_json(orient="records")

#Cities Data
CitiesDF =  dataframe_mysql_cities.toPandas()
CitiesJSON = CitiesDF.to_json(orient="records")


### Elasticsearch Index Creation for SQL Data  (2)

In [11]:
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200/')
es.indices.create(index="airports_api", ignore=400)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [airports_api/7eTuGTdOQae6f3he1cVfmQ] already exists',
    'index_uuid': '7eTuGTdOQae6f3he1cVfmQ',
    'index': 'airports_api'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [airports_api/7eTuGTdOQae6f3he1cVfmQ] already exists',
  'index_uuid': '7eTuGTdOQae6f3he1cVfmQ',
  'index': 'airports_api'},
 'status': 400}

In [12]:
es2 = Elasticsearch('http://localhost:9200/')
es2.indices.create(index="cities_api", ignore=400)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [cities_api/1E0f-IKOTjuQROKCVNoh7w] already exists',
    'index_uuid': '1E0f-IKOTjuQROKCVNoh7w',
    'index': 'cities_api'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [cities_api/1E0f-IKOTjuQROKCVNoh7w] already exists',
  'index_uuid': '1E0f-IKOTjuQROKCVNoh7w',
  'index': 'cities_api'},
 'status': 400}

### Parse Data to Index in ES (3)

In [13]:
import json

for i, item in AirportDF.iterrows():
    es.index(index="airports_api", 
         doc_type = "json" , 
         body = {"IATA":item['IATA'],"Airport_Name":item['Airport_Name'],"City_ID":item['City_ID']})

In [14]:
for x, items in CitiesDF.iterrows():
    es2.index(index="cities_api", 
         doc_type = "json", 
         body = {"City_ID":items['City_ID'],"City_Name":items[' City_Name'],"Country_Name":items[' Country_Name'],"Country_Code":items[' Country_Code']})